In [39]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, explained_variance_score

In [12]:
data=pd.read_csv("data/only_mood_preprocessed_dataset.csv")
#data=data.drop(1).reset_index()
#data=data.drop("index", axis=1)
data.head()

id day_to_predict  activity  appCat.builtin  appCat.communication  \
0  AS14.01     1900-03-24  0.171224       11.394910             53.411540   
1  AS14.01     1900-03-25  0.152861        8.299354             50.893593   
2  AS14.01     1900-03-26  0.106692        9.047925             56.807075   
3  AS14.01     1900-03-27  0.092881        7.823168             54.680243   
4  AS14.01     1900-03-28  0.119731        8.823213             53.284080   

   appCat.entertainment  appCat.finance  appCat.game  appCat.office  \
0             43.711556        8.023028          0.0            0.0   
1             66.744014        7.853472          0.0            0.0   
2             62.557681        9.099528          0.0            0.0   
3             95.430935        8.389889          0.0            0.0   
4             78.403032       11.186111          0.0            0.0   

   appCat.other       ...         appCat.unknown  appCat.utilities  \
0     18.708823       ...                    0.0         30.774000   
1     12.416865       ...                    0.0         52.886333   
2     17.677778       ...                    0.0         47.247100   
3     19.048611       ...                    0.0         48.340100   
4     18.602722       ...                    0.0         23.976555   

   appCat.weather  call  circumplex.arousal  circumplex.valence      mood  \
0             0.0   0.0            0.333333            0.500000  6.466667   
1             0.0   0.0            0.533333            0.400000  6.400000   
2             0.0   0.0            0.500000            0.433333  6.516667   
3             0.0   0.0            0.366667            0.366667  6.450000   
4             0.0   0.0            0.166667            0.633333  6.783333   

       screen  sms  mood_to_predict  
0   89.449056  0.0             6.00  
1   63.817831  0.0             6.75  
2  104.312375  0.0             6.60  
3  113.419714  0.0             7.00  
4  130.896018  0.0             6.40  

[5 rows x 22 columns]

In [42]:
def rfr_model(X_train, X_test, y_train, y_test):
    # Perform Grid-Search
    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (10, 50, 100, 1000),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
    grid_result = gsc.fit(X_train, y_train)
    best_params = grid_result.best_params_
    print(best_params)
    rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"], random_state=False, verbose=False)                            
    #scores = cross_val_score(rfr, X, y, cv=3, scoring='neg_mean_absolute_error')
    #predictions = cross_val_predict(rfr, X, y, cv=10)
    model=rfr.fit(X_train, y_train)
    print(model.feature_importances_)
    prediction=model.predict(X_test)
    return prediction

In [53]:
#id1= data.loc[data['id'] == "AS14.01"]
# y=id1.iloc[:,-1]
# x=id1.iloc[:,2:21]
y=data.iloc[:,-1]
x=data.iloc[:,2:21]

In [55]:

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30)
prediction=rfr_model(X_train, X_test, y_train, y_test)
count=0
for i in zip(y_test,prediction):
    if i[1]>i[0]-0.5 and i[1]<i[0]+0.5:
        count+=1
print("number correct prediction: "+str(count)+" out of "+str(len(prediction)-1))    
print(explained_variance_score(y_test,prediction))

C:\Users\david\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'max_depth': 3, 'n_estimators': 1000}
[1.49567714e-02 1.85868093e-02 2.57400224e-02 3.84253153e-02
 0.00000000e+00 3.67077454e-05 6.89020358e-04 6.44373544e-02
 4.33316515e-02 1.08245239e-02 3.07688521e-03 2.81729927e-04
 0.00000000e+00 5.59547329e-04 2.04810869e-02 2.22152074e-02
 7.22054769e-01 1.11721672e-02 3.13043040e-03]
number correct prediction: 215 out of 338
0.25986382962390187
